In [94]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

def plot_discrete_PDE(xs,ts,W,z_lims):
    '''
    Grafica en 3D la malla W de soluciones en los puntos dados por xs,ts.
    Los limites en z los da la tupla z_lims
    '''
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    X, T = np.meshgrid(xs, ts)
    surf = ax.plot_surface(X, T, W, rstride=1, cstride=1, linewidth=0, antialiased=False)
    ax.set_zlim(z_lims[0],z_lims[1])
    plt.show()
    
def pde_solver(h,k,T,c,f,g,period=False, a=0, b=0, l=0 ,r=0):
    '''
    Retorna una tupla con la malla de soluciones de la ecuación de onda, los puntos x y los puntos t usados.
    Los parametros estan definidos en el enunciado
    '''
    if period:
        return periodic_pde_sovlver(h,k,T,c,f,g)
    else:
        if a == b == 1:
            return dirichlet_pde_solver(h,k,T,c,f,g,l,r)
        elif a == b == 0:
            return neumann_pde_solver(h,k,T,c,f,g,l,r)
        elif a == 1 and b == 0:
            return mixed10_pde_solver(h,k,T,c,f,g,l,r)
        elif a == 0 and b == 1:
            return mixed01_pde_solver(h,k,T,c,f,g,l,r)
        else:
            return robin_pde_solver(h,k,T,c,f,g,a,b,l,r)
   
def initializer(h,k,T,c,f,g,l,r):
    '''
    Retorna una tupla con la matriz de soluciones aproximadas inicializada (con 
    las soluciones en el instante inicial y el segundo instante sin bordes ),
    el conjunto de puntos en x, el conjunto de puntos en t, sigma, A y las cantidades de puntos Nx_plus_1 y Nt_plus_1
    '''
    s = 1.0*c*k/h
    if(s>1):
        raise Exception("Inestable")
    xs = np.arange(-1,1+h,h)
    if(xs[-1]>1):
        xs = xs[:-1]
    ts = np.arange(0,T+k,k)
    if(ts[-1]>T):
        ts = ts[:-1]
    Nx_plus_1 = xs.shape[0]
    Nt_plus_1 = ts.shape[0]
    W = np.zeros((Nt_plus_1,Nx_plus_1))
    vec1 = np.array([g(xi) for xi in xs[1:-1]])
    vec2 = np.zeros(Nx_plus_1-2)
    vec2[0] = l(ts[0])
    vec2[-1] = r(ts[0])
    cons_vec = k*vec1+((s**2)/2)*vec2
    w0 = np.array([f(xi) for xi in xs[1:-1]])
    W[0,1:-1] = w0
    W[0,0] = f(xs[0])
    W[0,-1] = f(xs[-1])
    A = np.zeros((Nx_plus_1-2,Nx_plus_1-2))
    for i in range(Nx_plus_1-2):
        if i == 0:
            A[0,0] = 2-2*s**2
            A[0,1] = s**2
        elif i == Nx_plus_1-3:
            A[i,i-1] = s**2
            A[i,i] = 2-2*s**2
        else:
            A[i,i-1] = s**2
            A[i,i] = 2-2*s**2
            A[i,i+1] = s**2
    W[1,1:-1] = (1/2)*(np.dot(A,w0)) + cons_vec
    return W,xs,ts,s,A,Nx_plus_1,Nt_plus_1
    
def dirichlet_pde_solver(h,k,T,c,f,g,l,r):
    W,xs,ts,s,A,Nx_plus_1,Nt_plus_1 = initializer(h,k,T,c,f,g,l,r)
    W[:,0] = np.array([l(t) for t in ts]) #Se conocen los valores de la funcion en borde izquierdo 
    W[:,-1] = np.array([r(t) for t in ts]) #Se conocen los valores de la funcion en borde derecho
    for i in range(2,Nt_plus_1):
        vec = np.zeros(Nx_plus_1-2)
        vec[0] = l(ts[i-1])
        vec[-1] = r(ts[i-1])
        wk_1 = W[i-1,1:-1]
        wk_2 = W[i-2,1:-1]
        W[i,1:-1] = np.dot(A,wk_1) - wk_2 + s**2*vec
    return W,xs,ts
                
def evaluated_interpolation_constants(xs):
    '''
    Retorna 6 valores usados en la interpolación polinomial de los tres primeros y ultimos
    puntos del dominio espacial del problema, para cada instante.
    L'(-1) = l1 * w_0,k + l2 * w1,k + l3 * w_2,k
    L'(1) =  l4 * w_Nx-2,k + l5 * w_Nx-1,k + l6 * w_Nx,k
    Estos valores son usados para despejar posteriormente los valores de la funcion en la frontera
    '''
    x1 = xs[1]
    x2 = xs[2]
    l1 = (-2-x1-x2)/((-1-x1)*(-1-x2))
    l2 = (-1-x2)/((x1+1)*(x1-x2))
    l3 = (-1-x1)/((x2+1)*(x2-x1))
    xNx_1 = xs[-2]
    xNx_2 = xs[-3]
    l4 = (2-xNx_2-xNx_1)/((1-xNx_2)*(1-xNx_1))
    l5 = (1-xNx_2)/((xNx_1-xNx_2)*(xNx_1-1))
    l6 = (1-xNx_1)/((xNx_2-xNx_1)*(xNx_2-1))
    return l1,l2,l3,l4,l5,l6
    
def neumann_pde_solver(h,k,T,c,f,g,l,r):
    W,xs,ts,s,A,Nx_plus_1,Nt_plus_1 = initializer(h,k,T,c,f,g,l,r)
    l1,l2,l3,l4,l5,l6 = evaluated_interpolation_constants(xs)
    for i in range(2,Nt_plus_1):
        vec = np.zeros(Nx_plus_1-2)
        vec[0] = l(ts[i-1])
        vec[-1] = r(ts[i-1])
        wk_1 = W[i-1,1:-1]
        wk_2 = W[i-2,1:-1]
        W[i,1:-1] = np.dot(A,wk_1) - wk_2 + s**2*vec
        ltk = l(ts[i])
        W[i,0]  = (1/l1)*(ltk - W[i,1]*l2 - W[i,2]*l3) #aproximación del borde, que surge del uso del polinomio interpolador
        rtk = r(ts[i])
        W[i,-1] = (1/l6)*(rtk - W[i,-2]*l5 - W[i,-3]*l4) #aproximación del borde, que surge del uso del polinomio interpolador
    return W,xs,ts

def robin_pde_solver(h,k,T,c,f,g,a,b,l,r):
    W,xs,ts,s,A,Nx_plus_1,Nt_plus_1 = initializer(h,k,T,c,f,g,l,r)
    l1,l2,l3,l4,l5,l6 = evaluated_interpolation_constants(xs)
    for i in range(2,Nt_plus_1):
        vec = np.zeros(Nx_plus_1-2)
        vec[0] = l(ts[i-1])
        vec[-1] = r(ts[i-1])
        wk_1 = W[i-1,1:-1]
        wk_2 = W[i-2,1:-1]
        W[i,1:-1] = np.dot(A,wk_1) - wk_2 + s**2*vec
        ltk = l(ts[i])
        W[i,0]  = (ltk - (1-a)*(W[i,1]*l2 + W[i,2]*l3))/(a+(1-a)*l1) #aproximación del borde, que surge del uso del polinomio interpolador
        rtk = r(ts[i])
        W[i,-1] = (rtk - (1-b)*(W[i,-3]*l4 + W[i,-2]*l5))/(b+(1-b)*l6) #aproximación del borde, que surge del uso del polinomio interpolador
    return W,xs,ts

def mixed10_pde_solver(h,k,T,c,f,g,l,r):
    W,xs,ts,s,A,Nx_plus_1,Nt_plus_1 = initializer(h,k,T,c,f,g,l,r)
    _,_,_,l4,l5,l6 = evaluated_interpolation_constants(xs)
    W[:,0] = np.array([l(t) for t in ts]) #Se conocen los valores de la funcion en borde izquierdo
    for i in range(2,Nt_plus_1):
        vec = np.zeros(Nx_plus_1-2)
        vec[0] = l(ts[i-1])
        vec[-1] = r(ts[i-1])
        wk_1 = W[i-1,1:-1]
        wk_2 = W[i-2,1:-1]
        W[i,1:-1] = np.dot(A,wk_1) - wk_2 + s**2*vec
        rtk = r(ts[i])
        W[i,-1] = (1/l4)*(rtk - W[i,-2]*l5 - W[i,-3]*l6) #aproximación del borde, que surge del uso del polinomio interpolador
    return W,xs,ts

In [77]:
np.array([1,2,3])[:-1]

array([1, 2])

2.- 

In [96]:
T = 10.0
c = 1
k = T/49
h = 2.0/9

def f(x):
    return 100*np.sin(x**2)

def g(x):
    return 200*x*np.cos(x**2)

def l(t):
    return 0

def r(t):
    return 0
W,xs,ts = pde_solver(h,k,T,c,f,g,a=1, b=1, l=l ,r=r)
plot_discrete_PDE(xs,ts,W,(-300,300))
print W
'''
T = 10.0
c = 1
k = T/100
h = 2.0/10
'''
W,xs,ts = pde_solver(h,k,T,c,f,g,a=0, b=0, l=l ,r=r)
plot_discrete_PDE(xs,ts,W,(-300,300))
print W
W,xs,ts = pde_solver(h,k,T,c,f,g,a=0.5, b=0.5, l=l ,r=r)
plot_discrete_PDE(xs,ts,W,(-300,300))
print W

[[  0.00000000e+00   5.68711303e+01   3.03765063e+01   1.10882629e+01
    1.23453654e+00   1.23453654e+00   1.10882629e+01   3.03765063e+01
    5.68711303e+01   0.00000000e+00]
 [  0.00000000e+00  -2.61122922e+01  -2.16042397e+01  -1.35215443e+01
   -4.53480178e+00   4.53480178e+00   1.35215443e+01   2.16042397e+01
    2.61122922e+01   0.00000000e+00]
 [  0.00000000e+00  -8.32705594e+01  -7.05701370e+01  -3.73688794e+01
   -1.02342552e+01   7.76518216e+00   1.51923537e+01   9.81712436e+00
   -3.04717013e+01   0.00000000e+00]
 [  0.00000000e+00  -5.94870364e+01  -1.02245659e+02  -6.63328045e+01
   -2.36383122e+01   2.07892845e+00   6.06563629e+00  -3.14160687e+01
   -2.73763666e+01   0.00000000e+00]
 [  0.00000000e+00  -2.15947928e+01  -6.75697686e+01  -8.95771059e+01
   -5.13609592e+01  -2.19348259e+01  -3.80354817e+01  -3.76301658e+01
   -4.59892206e+00   0.00000000e+00]
 [  0.00000000e+00  -4.26476103e+00  -1.26796059e+01  -6.20290061e+01
   -8.64970910e+01  -8.43458097e+01  -6.82154